# <font color='blue'>Multilayer Perceptron</font>

In [99]:
import numpy as np

class Node:
    def __init__(self, inputs = []):
        self.inputs = inputs
        self.outputs = []
        self.value = None
        self.gradients = {}
        for n in inputs:
            n.outputs.append(self)

    def forward(self):
        raise NotImplementedError

    def backward(self):
        raise NotImplementedError
        
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)


class Input(Node):
    def __init__(self):
        Node.__init__(self)

    def forward(self):
        pass

    def backward(self):
        self.gradients = {self: 0}
        for n in self.outputs:
            self.gradients[self] = self.gradients[self] + n.gradients[self]
        pass
    
class Transfer(Node):
    def __init__(self, x, w, b):
        Node.__init__(self, [x, w, b])

    def forward(self):
        x = self.inputs[0].value
        w = self.inputs[1].value
        b = self.inputs[2].value
        self.value = np.dot(x, w) + b

    def backward(self):
        self.gradients = {n: np.zeros_like(n.value) for n in self.inputs}
        
        for n in self.outputs:
            # x
            self.gradients[self.inputs[0]] = self.gradients[self.inputs[0]] + np.dot(n.gradients[self], self.inputs[1].value.T)
            # w
            self.gradients[self.inputs[1]] = self.gradients[self.inputs[1]] + np.dot(self.inputs[0].value.T, n.gradients[self])
            # b
            self.gradients[self.inputs[2]] = self.gradients[self.inputs[2]] + np.sum(n.gradients[self], axis = 0, keepdims = False)
            
class Activation(Node):
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, z):
        return 1. / (1. + np.exp(-z))
    
    def forward(self):
        self.value = self._sigmoid(self.inputs[0].value)

    def backward(self):
        ## self.inputs == Tranfer Object
        self.gradients = {n: np.zeros_like(n.value) for n in self.inputs}
        for n in self.outputs:
            sigmoid = self.value
            self.gradients[self.inputs[0]] = self.gradients[self.inputs[0]] + sigmoid * (1 - sigmoid) * n.gradients[self]
    
class Error(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y, a])
   
    def forward(self):
        y = self.inputs[0].value.reshape(-1, 1)
        a = self.inputs[1].value.reshape(-1, 1)
        self.value = np.mean((y - a)**2)
        self.m = y.shape[0]
        self.diff = y - a
        
        
    def backward(self):
        self.gradients[self.inputs[0]] = (2 / self.m) * self.diff
        self.gradients[self.inputs[1]] = (-2 / self.m) * self.diff
    

def train(data_train, data_test, epochs, batch_size, steps_per_epoch):
    data = initialize(data_train, data_test)
    for i in range(epochs):
        loss = 0
        for j in range(steps_per_epoch):
            data_train_batch, data_test_batch = resample(data_train, data_test, n_samples = batch_size)
            X.value = data_train_batch
            y.value = data_test_batch
            
            sorted_nodes = sort(data)
            for n in sorted_nodes:
                n.forward()

            for n in sorted_nodes[::-1]:
                n.backward()

            sgd_update([W1, b1, W2, b2])
    
        print("Epoch: {}, Custo: {:.3f}".format(i+1, sorted_nodes[-1].value/steps_per_epoch))


def sequencial():
    X, y = Input(), Input()
    W1, b1 = Input(), Input()
    W2, b2 = Input(), Input()

def dense():
    l1 = Transfer(X, W1, b1)
    s1 = Activation(l1)
    l2 = Transfer(s1, W2, b2)
    cost = Error(y, l2)
    
def initialize(X_, y_):
    W1_ = np.random.randn(n_features, n_hidden)
    b1_ = np.zeros(n_hidden)
    W2_ = np.random.randn(n_hidden, 1)
    b2_ = np.zeros(1)
    
    return {
        X: X_,
        y: y_,
        W1: W1_,
        b1: b1_,
        W2: W2_,
        b2: b2_
    }
    
# using kahn's algorithm
def sort(data):
    
    #get the input class nodes
    inputs = [n for n in data.keys()]
    nodes = [n for n in inputs]
    queue = {}
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in queue:
            queue[n] = {'in': set(), 'out': set()}
        for m in n.outputs:
            if m not in queue:
                queue[m] = {'in': set(), 'out': set()}
            queue[n]['out'].add(m)
            queue[m]['in'].add(n)
            nodes.append(m)
    G = []
    TS = set(inputs)
    while len(TS) > 0:
        n = TS.pop()

        if isinstance(n, Input):
            n.value = data[n]

        G.append(n)
        for m in n.outputs:
            queue[n]['out'].remove(m)
            queue[m]['in'].remove(n)
            if len(queue[m]['in']) == 0:
                TS.add(m)
    return G
   
def sgd_update(params, learning_rate = 1e-2):
    for t in params:
        partial = t.gradients[t]
        t.value -= learning_rate * partial

In [100]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

data = load_boston()

data_train = data['data']
data_test = data['target']

data_train = (data_train - np.mean(data_train, axis = 0)) / np.std(data_train, axis = 0)

n_features = data_train.shape[1]
n_hidden = 20

epochs = 1000
m = data_train.shape[0]

batch_size = 11
steps_per_epoch = m // batch_size

sequencial()
train(data_train, data_test,epochs, batch_size, steps_per_epoch)

Epoch: 1, Custo: 0.034
Epoch: 2, Custo: 0.027
Epoch: 3, Custo: 0.242
Epoch: 4, Custo: 0.064
Epoch: 5, Custo: 0.048
Epoch: 6, Custo: 0.129
Epoch: 7, Custo: 0.025
Epoch: 8, Custo: 0.054
Epoch: 9, Custo: 0.043
Epoch: 10, Custo: 0.076
Epoch: 11, Custo: 0.062
Epoch: 12, Custo: 0.028
Epoch: 13, Custo: 0.051
Epoch: 14, Custo: 0.231
Epoch: 15, Custo: 0.035
Epoch: 16, Custo: 0.043
Epoch: 17, Custo: 0.065
Epoch: 18, Custo: 0.151
Epoch: 19, Custo: 0.073
Epoch: 20, Custo: 0.046
Epoch: 21, Custo: 0.047
Epoch: 22, Custo: 0.067
Epoch: 23, Custo: 0.068
Epoch: 24, Custo: 0.220
Epoch: 25, Custo: 0.080
Epoch: 26, Custo: 0.036
Epoch: 27, Custo: 0.043
Epoch: 28, Custo: 0.129
Epoch: 29, Custo: 0.029
Epoch: 30, Custo: 0.121
Epoch: 31, Custo: 0.069
Epoch: 32, Custo: 0.104
Epoch: 33, Custo: 0.033
Epoch: 34, Custo: 0.024
Epoch: 35, Custo: 0.060
Epoch: 36, Custo: 0.031
Epoch: 37, Custo: 0.028
Epoch: 38, Custo: 0.011
Epoch: 39, Custo: 0.033
Epoch: 40, Custo: 0.015
Epoch: 41, Custo: 0.110
Epoch: 42, Custo: 0.040
E

Epoch: 344, Custo: 0.046
Epoch: 345, Custo: 0.053
Epoch: 346, Custo: 0.029
Epoch: 347, Custo: 0.090
Epoch: 348, Custo: 0.042
Epoch: 349, Custo: 0.035
Epoch: 350, Custo: 0.052
Epoch: 351, Custo: 0.048
Epoch: 352, Custo: 0.055
Epoch: 353, Custo: 0.070
Epoch: 354, Custo: 0.152
Epoch: 355, Custo: 0.040
Epoch: 356, Custo: 0.017
Epoch: 357, Custo: 0.105
Epoch: 358, Custo: 0.035
Epoch: 359, Custo: 0.037
Epoch: 360, Custo: 0.087
Epoch: 361, Custo: 0.042
Epoch: 362, Custo: 0.051
Epoch: 363, Custo: 0.027
Epoch: 364, Custo: 0.033
Epoch: 365, Custo: 0.090
Epoch: 366, Custo: 0.026
Epoch: 367, Custo: 0.046
Epoch: 368, Custo: 0.023
Epoch: 369, Custo: 0.024
Epoch: 370, Custo: 0.028
Epoch: 371, Custo: 0.135
Epoch: 372, Custo: 0.031
Epoch: 373, Custo: 0.040
Epoch: 374, Custo: 0.017
Epoch: 375, Custo: 0.033
Epoch: 376, Custo: 0.033
Epoch: 377, Custo: 0.051
Epoch: 378, Custo: 0.092
Epoch: 379, Custo: 0.020
Epoch: 380, Custo: 0.191
Epoch: 381, Custo: 0.067
Epoch: 382, Custo: 0.032
Epoch: 383, Custo: 0.059


Epoch: 676, Custo: 0.033
Epoch: 677, Custo: 0.047
Epoch: 678, Custo: 0.037
Epoch: 679, Custo: 0.070
Epoch: 680, Custo: 0.019
Epoch: 681, Custo: 0.043
Epoch: 682, Custo: 0.057
Epoch: 683, Custo: 0.021
Epoch: 684, Custo: 0.051
Epoch: 685, Custo: 0.041
Epoch: 686, Custo: 0.017
Epoch: 687, Custo: 0.037
Epoch: 688, Custo: 0.029
Epoch: 689, Custo: 0.040
Epoch: 690, Custo: 0.074
Epoch: 691, Custo: 0.062
Epoch: 692, Custo: 0.030
Epoch: 693, Custo: 0.016
Epoch: 694, Custo: 0.058
Epoch: 695, Custo: 0.037
Epoch: 696, Custo: 0.027
Epoch: 697, Custo: 0.035
Epoch: 698, Custo: 0.030
Epoch: 699, Custo: 0.106
Epoch: 700, Custo: 0.037
Epoch: 701, Custo: 0.066
Epoch: 702, Custo: 0.026
Epoch: 703, Custo: 0.055
Epoch: 704, Custo: 0.031
Epoch: 705, Custo: 0.032
Epoch: 706, Custo: 0.026
Epoch: 707, Custo: 0.057
Epoch: 708, Custo: 0.015
Epoch: 709, Custo: 0.077
Epoch: 710, Custo: 0.053
Epoch: 711, Custo: 0.077
Epoch: 712, Custo: 0.117
Epoch: 713, Custo: 0.026
Epoch: 714, Custo: 0.021
Epoch: 715, Custo: 0.039
